In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import datetime
import logging

import dataflow.system as dtfsys
import market_data as mdata
import helpers.hdbg as hdbg
import helpers.henv as henv
import helpers.hprint as hprint
import numpy as np
import pandas as pd
import sklearn.linear_model as sklimod
import sklearn.model_selection as skmosel

In [ ]:
hdbg.init_logger(verbosity=logging.DEBUG)

_LOG = logging.getLogger(__name__)

_LOG.info("%s", henv.get_system_signature()[0])

hprint.config_notebook()

In [ ]:
import pandas as pd
import pytest

import im_v2.ccxt.data.client as icdcl
import im_v2.common.data.client as icdc
import im_v2.common.data.client.test.im_client_test_case as icdctictc
import im_v2.crypto_chassis.data.client as iccdc


# https://github.com/cryptokaizen/cmamp/pull/2315


# Initialize the client.
universe_version = "v4"
resample_1min = False
dataset = "ohlcv"
contract_type = "futures"
data_snapshot = "20220620"
im_client = icdcl.get_CcxtHistoricalPqByTileClient_example1(
    universe_version,
    resample_1min,
    dataset,
    contract_type,
    data_snapshot,
)
# Set expected values.
full_symbols = ["binance::BTC_USDT", "binance::ADA_USDT"]
start_ts = pd.Timestamp("2022-05-01 13:00:00+00:00")
end_ts = pd.Timestamp("2022-05-01 13:05:00+00:00")

columns = None
filter_data_mode = "assert"
actual_df = im_client.read_data(
    full_symbols, start_ts, end_ts, columns, filter_data_mode
)

In [ ]:
symbols = im_client.get_universe()[3:6]
display(symbols)

In [ ]:
asset_ids = im_client.get_asset_ids_from_full_symbols(symbols)
print(asset_ids)

In [ ]:
import market_data.market_data_example as mdmadaex

#asset_ids = None
columns = None
column_remap = None

market_data = mdmadaex.get_HistoricalImClientMarketData_example1(
    im_client,
    asset_ids,
    columns,
    column_remap)

In [ ]:
start_ts = pd.Timestamp("2022-05-01 13:00:00+00:00")
end_ts = pd.Timestamp("2022-05-01 13:05:00+00:00")
ts_col_name = "knowledge_timestamp"
        
df = market_data.get_data_for_interval(
    start_ts,
    end_ts,
    ts_col_name,
    asset_ids,
)

In [ ]:
df.tail()

In [ ]:
data_source_node = dtfsys.HistoricalDataSource(
    "load_prices",
    market_data,
    "end_ts",
    True,
    col_names_to_remove=["knowledge_timestamp", "start_ts", "full_symbol"],
)

In [ ]:
data = data_source_node.fit()["df_out"]

In [ ]:
data.tail()